In [1]:
import math

def calculate_angles(p1, p2, p3):
    """
    計算由三個座標形成的三角形的三個角度。

    參數：
    p1, p2, p3: tuple，代表三個座標 (x, y)

    返回：
    tuple，三角形的三個角度 (角1, 角2, 角3)，單位為度。
    """
    def distance(a, b):
        # 計算兩點之間的距離
        return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

    # 計算三邊長
    a = distance(p2, p3)  # 邊a對應角點p1
    b = distance(p1, p3)  # 邊b對應角點p2
    c = distance(p1, p2)  # 邊c對應角點p3

    # 使用餘弦定理計算角度
    angle1 = math.acos((b**2 + c**2 - a**2) / (2 * b * c))  # p1的角
    angle2 = math.acos((a**2 + c**2 - b**2) / (2 * a * c))  # p2的角
    angle3 = math.acos((a**2 + b**2 - c**2) / (2 * a * b))  # p3的角

    # 將角度從弧度轉換為度
    angle1 = math.degrees(angle1)
    angle2 = math.degrees(angle2)
    angle3 = math.degrees(angle3)

    return angle1, angle2, angle3 ,a , b , c


In [2]:
p1 = (1807,1897)
p2 = (1807,1119)
p3 = (1497,491)
print(calculate_angles(p1, p2, p3))

(12.433847185185085, 153.72764502265716, 13.838507792157724, 700.3456289575884, 1439.7694259845914, 778.0)


In [4]:

def ptg(p1,p2,p3):
    angle1, angle2, angle3 ,len1 , len2 , len3 = calculate_angles(p1, p2, p3)

    grade = 0
    grade += (1 - (len3/len1 - 778.0/700.3456289575884)**2**.5) * 30
    grade += (1 - (len3/len2 - 778.0/1439.7694259845914)**2**.5) * 30
    grade += (1 - (1 - angle1/12.433847185185085)**2**.5) * 15
    grade += (1 - (1 - angle2/153.72764502265716)**2**.5) * 15
    grade += (1 - (1 - angle3/13.838507792157724)**2**.5) * 10

    if  len2 < len1:
        grade = 0

    
    return grade


print(ptg(p2,p1,p3))


0


# load model

In [27]:
import torch.nn as nn
import torch.nn.functional as F

class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [28]:
import torch

# 初始化模型
model_1 = CNNPointDetector()
model_1.load_state_dict(torch.load("fan_corner1_detector_1127.pth"))
model_1.eval()  # 切换到评估模式
print("Model_1 loaded and ready for inference.")

model_2 = CNNPointDetector()
model_2.load_state_dict(torch.load("fan_corner2_detector_1127.pth"))
model_2.eval()  # 切换到评估模式
print("Model_2 loaded and ready for inference.")

model_3 = CNNPointDetector()
model_3.load_state_dict(torch.load("fan_corner3_detector_1127.pth"))
model_3.eval()  # 切换到评估模式
print("Model_3 loaded and ready for inference.")

Model_1 loaded and ready for inference.
Model_2 loaded and ready for inference.
Model_3 loaded and ready for inference.


C:\Users\raykuo7\AppData\Local\Temp\ipykernel_13876\1465348596.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_1.load_state_dict(torch.load("fan_corner1_detector_1

In [ ]:
import sys
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import glob
import os
import numpy as np

# image_files = glob.glob(r'C:\GitHub\fantest_pic\try1128\*.png')

# if not image_files:
#     print("No images found. Please check the path and try again.")
#     sys.exit()

class FanDatasetFromPath(Dataset):
    def __init__(self, image_folder, img_size=64):
        """
        Args:
            image_folder (str): 路径到包含图片的文件夹
            img_size (int): 调整图像大小的目标尺寸
        """
        self.image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.img_size = img_size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        # 读取图片并转为灰度图
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 255.0  # Resize and normalize to [0, 1]
        image = image.reshape(1, self.img_size, self.img_size)  # Add channel dimension

        # 标签设为零向量（根据需求调整）
        label = np.zeros(2, dtype=np.float32)  # Placeholder for labels (e.g., [x, y] coordinates)

        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用图片路径加载数据集
image_folder_path = r'C:\GitHub\fantest_pic\try1128'  # 替换为你的图片文件夹路径
dataset = FanDatasetFromPath(image_folder_path)

with torch.no_grad():
    for i in range(len(dataset)):
            img, label = dataset[i]
            img = img.unsqueeze(0)  # Add batch dimension


            p1 = model_1(img)
            p2 = model_2(img)
            p3 = model_3(img)

            p1 = (p1.numpy().flatten()[0] , p1.numpy().flatten()[1])
            p2 = (p2.numpy().flatten()[0] , p2.numpy().flatten()[1])
            p3 = (p3.numpy().flatten()[0] , p3.numpy().flatten()[1])
    print(img , "Grade: ", ptg(p1,p2,p3))
    
            
# for image_path in image_files:
    # image = cv2.imread(image_path)

    



    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # transform = transforms.Compose([
    # transforms.ToPILImage(),
    # transforms.Resize((2047, 2047)),
    # transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5], std=[0.5])  # 单通道归一化
    # ])

    # transform = transforms.Compose([
    # transforms.ToPILImage(),
    # transforms.Resize((2047, 2047)),  # 替换为你的模型输入尺寸
    # transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 如果训练时有不同的归一化设置，请替换
    # ])

    # input_tensor = transform(image)
    # input_tensor = input_tensor.unsqueeze(0) # 增加 batch 维度

                # with torch.no_grad():  # 不需要梯度计算


tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]]) Grade:  0


: 

In [ ]:
import os

def add_prefix_to_files(folder_path, prefix="1_"):
    """
    將指定資料夾內所有檔案的名稱前加上指定的前綴。

    參數：
        folder_path (str): 資料夾的路徑。
        prefix (str): 要添加到檔名前的字串，預設為 "_1"。
    """
    try:
        # 確認資料夾存在
        if not os.path.exists(folder_path):
            print(f"資料夾不存在: {folder_path}")
            return

        # 遍歷資料夾內的所有檔案
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            # 確保只處理檔案，忽略資料夾
            if os.path.isfile(file_path):
                new_name = prefix + filename
                new_path = os.path.join(folder_path, new_name)

                # 重命名檔案
                os.rename(file_path, new_path)
                print(f"重命名: {filename} -> {new_name}")

        print("檔案重命名完成！")

    except Exception as e:
        print(f"發生錯誤: {e}")

# 使用範例
folder_path = "C:/GitHub/fantest_pic/temp"  # 替換成目標資料夾路徑
add_prefix_to_files(folder_path)
